#### Sources
https://www.researchgate.net/publication/236027587_Hierarchical_Color_Quantization_Based_on_Self-organization

http://blog.yhat.com/posts/self-organizing-maps-2.html

https://visualstudiomagazine.com/articles/2019/01/01/self-organizing-maps-python.aspx

https://rubikscode.net/2018/08/27/implementing-self-organizing-maps-with-python-and-tensorflow/

https://github.com/JustGlowing/minisom

https://medium.com/machine-learning-researcher/self-organizing-map-som-c296561e2117

https://github.com/davidasboth/blog-notebooks/blob/master/self-organising-map/Self-Organising%20Map.ipynb

https://github.com/davidasboth/blog-notebooks/blob/master/self-organising-map/Self-Organising%20Map.ipynb

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
import numpy as np

TensorFlow 1.x selected.


In [ ]:
Input = tf.placeholder(dtype='float', shape=[None, 2], name='Input')
weights = tf.Variable(initial_value=tf.random_normal(shape=[2,3], stddev=0.4, dtype='float', name='hiddenweights'))
input_bias = tf.Variable(initial_value=tf.random_normal(shape=[3], stddev=0.4, dtype='float', name='inputbias'))
hidden_bias = tf.Variable(initial_value=tf.random_normal(shape=[1], stddev=0.4, dtype='float', name='hiddenbias'))
output = tf.Variable(initial_value=tf.random_normal(shape=[3, 1], stddev=0.4, dtype='float', name='outputweights'))
Target = tf.placeholder(dtype='float', shape=[None, 1], name='Target')

hiddenLayer = tf.matmul(Input, weights) + input_bias
hiddenLayer = tf.sigmoid(hiddenLayer, name='hidden_layer_activaction')

outputLayer = tf.matmul(hiddenLayer, output) + hidden_bias
outputLayer = tf.sigmoid(outputLayer, name='output_layer_activaction')

cost = tf.squared_difference(Target, outputLayer) 
cost = tf.reduce_mean(cost)

optimizer = tf.train.AdamOptimizer().minimize(cost)

inp = [[1.,1.], [1.,0.], [0.,1.], [0.,0.]]
out = [[0], [1], [1], [0]]
epochs = 4000

with tf.Session() as session:
  tf.global_variables_initializer().run()
  for i in range(epochs):
    err, _ = session.run([cost, optimizer], feed_dict={Input: inp, Target: out})
    print(i, err)

In [ ]:
# SOM
input_matrix = [[0.,0.,1.,1.], [1.,0.,0.,0.]]
data_size = [2,4]
epochs = 10

learning_rate = tf.Variable(0.5)
input_data = tf.placeholder(dtype='float', shape=[None, data_size[1]], name='input_data')
weights = tf.Variable(initial_value=tf.random_normal(shape=[data_size[1], data_size[0]], stddev=0.4, dtype='float', name='weights'))
ROW = tf.placeholder(dtype='float', shape=[data_size[1]], name='ROW')
input_vector = tf.transpose(ROW)
tile = tf.reshape(tf.tile(input_vector, [data_size[0]]), [data_size[1], data_size[0]])
squared_diff = tf.squared_difference(weights, tile) # squared difference itemwise
d = tf.math.reduce_sum(squared_diff, axis=0) # sum along axis 0
closest_sum_index = tf.math.argmax(d) # choose smallest squared difference sum along columns
updated_col = weights[:,closest_sum_index] + learning_rate * (input_vector - weights[:,closest_sum_index]) # create updated vector
update_weights = weights[:,closest_sum_index].assign(updated_col) # update weights

with tf.Session() as session:
  tf.global_variables_initializer().run()
  data = session.run(input_data, {input_data: input_matrix})
  for _ in range(epochs):
    for row in data:
      session.run(update_weights, {ROW: row})
      print(weights.eval())

[[-0.2233635   0.6293827 ]
 [ 0.3241031   0.18949364]
 [ 0.27314147  0.57103515]
 [-0.18833636  0.15753645]]
[[ 0.38831824  0.6293827 ]
 [ 0.16205154  0.18949364]
 [ 0.13657074  0.57103515]
 [-0.09416818  0.15753645]]
[[ 0.38831824  0.31469136]
 [ 0.16205154  0.09474682]
 [ 0.13657074  0.7855176 ]
 [-0.09416818  0.57876825]]
[[ 0.69415915  0.31469136]
 [ 0.08102577  0.09474682]
 [ 0.06828537  0.7855176 ]
 [-0.04708409  0.57876825]]
[[0.34707958 0.31469136]
 [0.04051289 0.09474682]
 [0.5341427  0.7855176 ]
 [0.47645795 0.57876825]]
[[0.34707958 0.65734565]
 [0.04051289 0.04737341]
 [0.5341427  0.3927588 ]
 [0.47645795 0.28938413]]
[[0.34707958 0.32867283]
 [0.04051289 0.02368671]
 [0.5341427  0.6963794 ]
 [0.47645795 0.64469206]]
[[0.34707958 0.66433644]
 [0.04051289 0.01184335]
 [0.5341427  0.3481897 ]
 [0.47645795 0.32234603]]
[[0.34707958 0.33216822]
 [0.04051289 0.00592168]
 [0.5341427  0.67409486]
 [0.47645795 0.661173  ]]
[[0.34707958 0.6660841 ]
 [0.04051289 0.00296084]
 [0.53414

In [ ]:
# SOM 3D
input_matrix = np.matrix([[0.,0.,1.,1.], [1.,0.,0.,0.], [1.,0.,0.,0.]], dtype=tf.float64)
map_dimension = [5,6]
data_size = [3,4]
t = tf.Variable(1, dtype=tf.int64)
learning_rate = tf.Variable(0.5, dtype=tf.float64)
input_data = tf.placeholder(dtype=tf.float64, shape=[None, 4])
weights = tf.Variable(initial_value=tf.random_normal(shape=[4, 5, 6], stddev=0.4, dtype=tf.float64, seed=0))
input_vector = input_data[0,:]
input_vector_3d = tf.expand_dims(tf.expand_dims(input_vector, [1]), [1])
input_vector_multiplied = tf.tile(input_vector_3d, [1,5,6])
squared_diff = tf.squared_difference(weights, input_vector_multiplied)
distances = tf.math.reduce_sum(squared_diff, axis=0) # Euclidean distances but skip sqrt
closest_sum_indices = tf.transpose(tf.where(tf.equal(tf.math.reduce_min(distances), distances)))
closest_sum_weights_3d = tf.expand_dims(closest_sum_indices, [1])
closest_sum_weights_multiplied = tf.tile(closest_sum_weights_3d, [1,5,6])

ind1 = tf.expand_dims(tf.tile(tf.expand_dims(tf.range(5), [1]), [1,6]), [0])
ind2 = tf.expand_dims(tf.transpose(tf.tile(tf.expand_dims(tf.range(6), [1]), [1,5])), [0])
output_indices = tf.concat([ind1, ind2], axis=0)
output_weights = tf.math.reduce_sum(tf.squared_difference(tf.cast(closest_sum_weights_multiplied, tf.float64), tf.cast(output_indices, tf.float64)), axis=0)
delta = tf.cast(t, tf.float64)
neighbourhood = tf.math.exp(-output_weights/(2*delta*delta))
neighbourhood_3d = tf.expand_dims(neighbourhood, 0)
neighbourhood_multiplied = tf.tile(neighbourhood_3d, [4,1,1])

new_weights = weights.assign(weights - learning_rate * delta * neighbourhood * (input_vector_multiplied - weights))

with tf.Session() as session:
  tf.global_variables_initializer().run()

  print(session.run(squared_diff, {input_data: input_matrix}))

  


In [ ]:
# SOM 3D
input_matrix = [[0.,0.,1.,1.], [1.,0.,0.,0.], [1.,0.,0.,0.]]
map_dimension = [5,6]
data_size = [3,4]
epochs = 1
t = tf.Variable(1, dtype=tf.int64)
t_increment = t.assign(t+1)
learning_rate = tf.Variable(0.5, dtype=tf.float64)
input_data = tf.placeholder(dtype=tf.float64, shape=[None, 4])
shuffle = tf.random.shuffle(input_data)
weights = tf.Variable(initial_value=tf.random_normal(shape=[4, 5, 6], stddev=0.4, dtype=tf.float64, seed=0))
ROW = tf.placeholder(dtype=tf.float64, shape=[data_size[1]])
input_vector = ROW
input_vector_3d = tf.expand_dims(tf.expand_dims(input_vector, [1]), [1])
squared_diff = tf.squared_difference(weights, input_vector_3d)
distances = tf.math.reduce_sum(squared_diff, axis=0) # Euclidean distances but skip sqrt
closest_sum_indices = tf.transpose(tf.where(tf.equal(tf.math.reduce_min(distances), distances)))
closest_sum_weights_3d = tf.expand_dims(closest_sum_indices, [1])

ind1 = tf.expand_dims(tf.tile(tf.expand_dims(tf.range(5), [1]), [1,6]), [0])
ind2 = tf.expand_dims(tf.transpose(tf.tile(tf.expand_dims(tf.range(6), [1]), [1,5])), [0])
output_indices = tf.concat([ind1, ind2], axis=0)
output_weights = tf.math.reduce_sum(tf.squared_difference(tf.cast(closest_sum_weights_3d, tf.float64), tf.cast(output_indices, tf.float64)), axis=0)
delta = tf.cast(t, tf.float64)
neighbourhood = tf.math.exp(-output_weights/(2*delta*delta))
neighbourhood_3d = tf.expand_dims(neighbourhood, 0)

update_weights = weights.assign(weights - learning_rate * delta * neighbourhood_3d * (input_vector_3d - weights))

out_distances = tf.math.reduce_sum(tf.squared_difference(input_vector_3d, weights), axis=0)
winner_indices = tf.squeeze(tf.where(tf.equal(tf.math.reduce_min(out_distances), out_distances))) # for one row

with tf.Session() as session:
  tf.global_variables_initializer().run()
  
  for _ in range(epochs):
    data = session.run(shuffle, {input_data: input_matrix})
    for row in data:
      session.run(update_weights, {ROW: row})
      session.run(t_increment)
  

  winners = np.empty([data_size[0],2], dtype=int)
  data = session.run(input_data, {input_data: input_matrix})
  for i, row in enumerate(data):
    winners[i] = session.run(winner_indices, {ROW: row})
  print(winners)


[[4 0]
 [0 5]
 [0 5]]


In [ ]:
# SOM 3D
input_matrix = [[0.,0.,1.,1.,0.], [1.,0.,0.,0., 0.], [1.,0.,0.,0.,0.]]
map_dimension = [30,30]
data_size = [3,5]
epochs = 1

t = tf.Variable(1, dtype=tf.int64)
t_increment = t.assign(t+1)
learning_rate = tf.Variable(0.5, dtype=tf.float64)
input_data = tf.placeholder(dtype=tf.float64, shape=[None, data_size[1]])
shuffle = tf.random.shuffle(input_data)
weights = tf.Variable(initial_value=tf.random_normal(shape=[data_size[1], map_dimension[0], map_dimension[1]], stddev=0.4, dtype=tf.float64, seed=0))
ROW = tf.placeholder(dtype=tf.float64, shape=[data_size[1]])

input_vector = ROW
input_vector_3d = tf.expand_dims(tf.expand_dims(input_vector, [1]), [2])
squared_diff = tf.squared_difference(weights, input_vector_3d)
distances = tf.math.reduce_sum(squared_diff, axis=0) # Euclidean distances but skip sqrt
closest_sum_indices = tf.transpose(tf.where(tf.equal(tf.math.reduce_min(distances), distances)))
closest_sum_weights_3d = tf.expand_dims(closest_sum_indices, [1])

ind1 = tf.expand_dims(tf.tile(tf.expand_dims(tf.range(map_dimension[0]), [1]), [1, map_dimension[1]]), [0])
ind2 = tf.expand_dims(tf.transpose(tf.tile(tf.expand_dims(tf.range(map_dimension[1]), [1]), [1,map_dimension[0]])), [0])
output_indices = tf.concat([ind1, ind2], axis=0)
output_weights = tf.math.reduce_sum(tf.squared_difference(tf.cast(closest_sum_weights_3d, tf.float64), tf.cast(output_indices, tf.float64)), axis=0)
delta = tf.cast(t, tf.float64)
neighbourhood = tf.math.exp(-output_weights/(2*delta*delta))
neighbourhood_3d = tf.expand_dims(neighbourhood, 0)
update_weights = weights.assign(weights - learning_rate * delta * neighbourhood_3d * (input_vector_3d - weights))

out_distances = tf.math.reduce_sum(tf.squared_difference(input_vector_3d, weights), axis=0)
winner_indices = tf.squeeze(tf.where(tf.equal(tf.math.reduce_min(out_distances), out_distances))) # for one row


with tf.Session() as session:
  tf.global_variables_initializer().run()

  for _ in range(epochs):
    data = session.run(shuffle, {input_data: input_matrix})
    #print(session.run(output_weights))
    for row in data:
      session.run(update_weights, {ROW: row})
      session.run(t_increment)
  

  winners = np.empty([data_size[0],2], dtype=int)
  data = session.run(input_data, {input_data: input_matrix})
  for i, row in enumerate(data):
    winners[i] = session.run(winner_indices, {ROW: row})
  print(winners)